In [135]:
import pandas as pd 
import numpy as np

# Let's change how printing the series works. I need to see all elements in the Series 
# source: http://stackoverflow.com/questions/19124601/is-there-a-way-to-pretty-print-the-entire-pandas-series-dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

jobs = pd.read_csv('stackoverflow_jobs.csv')

# Exploring data

In [136]:
print jobs.columns

Index([u'jobid', u'title', u'employer', u'location', u'salary', u'description',
       u'tags', u'url', u'date', u'weeknum'],
      dtype='object')


In [161]:
jobs.dtypes

jobid           int64
title          object
employer       object
location       object
salary         object
description    object
tags           object
url            object
date           object
weeknum         int64
equity           bool
currency       object
salary_low     object
salary_high    object
dtype: object

# Salary


## Replacing NA values with empty strings in the salary column

In [138]:
jobs.salary = jobs.salary.fillna('')

## Extracting equity  

In [141]:
jobs['equity'] = jobs['salary'].str.contains('Provides Equity')

## Extracting currency and high - low salary


Need to extract currency, salary_low and salary_high from salary field and copy it to their own columns.

Using regex here to capture parts of the salary field into three columns: 
    - currency will capture zero or more characters that are non digits
    - number_low captures one or more characters that are digits and spearators (currently only comma is used)
    - number high will capture all the numbers plus separators from the dash until the end of the string 

In [158]:
# salary = jobs.salary
salary = jobs.salary.map(lambda x: x.replace("Provides Equity","").replace("/","").strip())

sal = salary.str.extract('(?P<currency>[^\d]*)(?P<number_low>[\d,]+) - (?P<number_high>[\d,]+$)')

sal.number_low = sal.number_low.fillna(0)
sal.number_high = sal.number_high.fillna(0)
sal.currency = sal.currency.fillna('')

# mapping the new columns back
jobs['currency'] = sal.currency
jobs['salary_low'] = sal.number_low
jobs['salary_high'] = sal.number_high


# Location 

We need better location information, so we can do analysis by countries and cities. For this we need to extract country, state and city out of location column. But first let's remove the __na__ values from location column. 

Then use a lambda to split the location into individual fields.  

In [27]:
jobs.location = jobs.location.fillna('') # sometimes we have nothing in the location field. 

location_split = lambda x: pd.Series([i for i in x.split(',')])
locations = jobs['location'].apply(location_split)

locations.rename(columns={0:'city', 1: 'location_1', 2: 'location_2'},inplace=True)

## Fixing US locations

US locations seems to be special. They are in the form of _city, state_, we need this to be in form of _city, state, country_, so let's fix this first. 

If we have a US state in _location_1_ column then put _US_ in _location_2_. 


In [30]:
# Fixing US States
us_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

locations['location_1'] = locations['location_1'].str.strip()
locations.loc[locations['location_1'].isin(us_states),'location_2'] = "US"

## Filling the state and country columns

If in a row location_2 is null then location_1 contains the country of that location, if location_2 is not empty thne location_2 is going to be the country and location_1 will contain the state. 

In [51]:
# if location_2 is null then location_1 column has the country 
# if location_2 is not null then location_2 has the country and location_1 contains the state 
jobs['country'] = np.where(locations['location_2'].isnull(), locations['location_1'], locations['location_2'])
jobs['state'] = np.where(locations['location_2'].notnull(), locations['location_1'], '')

jobs['city'] = locations['city']

# filling na for country 
jobs.country = jobs.country.fillna('')

# stripping spaces from new columns
jobs['city'] = jobs['city'].str.strip()
jobs['country'] = jobs['country'].str.strip()

In [76]:
jobs['country'].value_counts()

US                        3534
Germany                   1217
UK                         762
Netherlands                331
Canada                     240
                           160
Australia                  149
France                     112
Sweden                     101
Switzerland                 93
Ireland                     92
Finland                     67
Spain                       65
Austria                     61
India                       48
Poland                      42
Israel                      36
Denmark                     33
Hungary                     28
Thailand                    28
Italy                       24
Indonesia                   21
Czech Republic              20
South Africa                20
Japan                       20
United Arab Emirates        19
Malta                       16
New Zealand                 16
China                       15
Iran                        15
                          ... 
Portugal                     7
Brazil  

Now we can see what countries are posting the most jobs. It seems that the US, Deutschland, Germany and the UK are the top countries. But wait. Aren't Germany and Deutschland are the same country? Let's fix this and some other countries with native names. 

In [91]:
jobs[jobs['country'].str.contains('Deutschland')] = 'Germany' # Deutschland -> Germany
jobs[jobs['country'].str.contains('Österreich')] = 'Austria' # Österreich -> Austria
jobs[jobs['country'].str.contains('Suisse')] = 'Switzerland' # Suisse -> Switzerland
jobs[jobs['country'].str.contains('Schweiz')] = 'Switzerland' # Schweiz -> Switzerland
jobs[jobs['country'].str.contains('Espagne')] = 'Spain' # Espagne -> Spain
jobs[jobs['country'].str.contains('République tchèque')] = 'Czech Republic' # République tchèque -> Czech Republic
jobs[jobs['country'].str.contains('Niederlande')] = 'Netherlands' # Niederlande -> Netherlands

jobs['country'].value_counts()

US                        3534
Germany                   1217
UK                         762
Netherlands                334
Canada                     240
                           160
Australia                  149
France                     112
Sweden                     101
Switzerland                 93
Ireland                     92
Finland                     67
Spain                       65
Austria                     61
India                       48
Poland                      42
Israel                      36
Denmark                     33
Thailand                    28
Hungary                     28
Italy                       24
Indonesia                   21
Czech Republic              20
South Africa                20
Japan                       20
United Arab Emirates        19
New Zealand                 16
Malta                       16
China                       15
Iran                        15
                          ... 
Romania                      8
Portugal